In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import altair as alt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
// Select the raw orders and grab the current status and when the order was created
SELECT status, extordercreatedat, totalprice from PIPE17.PIPE17_RAW.ORDERS where extordercreatedat >= DATEADD(day, -30, CURRENT_DATE) order by extordercreatedat desc

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
df = cell2.to_pandas()

# Convert extordercreatedat to datetime
df['EXTORDERCREATEDAT'] = pd.to_datetime(df['EXTORDERCREATEDAT'])

# Group by every 3 days and status, then count the occurrences
grouped_df = df.groupby([pd.Grouper(key='EXTORDERCREATEDAT', freq='3D'), 'STATUS']).size().reset_index(name='count')


# Create a bar plot
st.title("Order Aging Report")

# Create a bar chart using Altair
chart = alt.Chart(grouped_df).mark_bar(size=20).encode(
    x='EXTORDERCREATEDAT:T',
    y='count:Q',
    color='STATUS:N',
    tooltip=['EXTORDERCREATEDAT', 'STATUS', 'count']
).properties(
    title='Status Count Per 3 Days',
    width=800,
    height=400
)

# Plot using st.altair_chart
st.altair_chart(chart, use_container_width=True)





In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
df = cell2.to_pandas()

# Convert extordercreatedat to datetime
df['EXTORDERCREATEDAT'] = pd.to_datetime(df['EXTORDERCREATEDAT'])

# Group by 3-day periods and sum the total revenue
grouped_df = df.groupby(pd.Grouper(key='EXTORDERCREATEDAT', freq='3D')).agg({'TOTALPRICE': 'sum'}).reset_index()


# Create a bar plot
st.title("Revenue Report")

chart = alt.Chart(grouped_df).mark_bar(size=20).encode(  # Adjust the size parameter to control bar width
    x=alt.X('EXTORDERCREATEDAT:T', title='Date'),
    y=alt.Y('TOTALPRICE:Q', title='Total Revenue'),
    tooltip=['EXTORDERCREATEDAT', 'TOTALPRICE']
).properties(
    title='Total Revenue Per 3 Days',
    width=800,
    height=400
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

# Display the chart in Streamlit
st.title("Total Revenue Per 3 Days")
st.altair_chart(chart, use_container_width=True)